# How to stitch images with Laplacian pyramids

## Compiler parameters

Set my Jypyter environment for the use of OpenCV in a C++ notebook. You don't need this line when you write yur own C++ programs. I need it to set my interactive compiler ([Cling](https://root.cern/cling/)). For your own program, use CMake.

In [1]:
#include "../../../includeLibraries.h"

## Header inclusion for C++

In [2]:
#include <iostream>
#include <vector>
#include <stdexcept>
#include <sstream>
#include <string>
#include <opencv2/opencv.hpp>

## Add the namespaces

In [3]:
using namespace std;

In [4]:
using namespace cv;

## Create a structure

to store the pyramid.

In [5]:
vector<Mat> p_laplacian_pyramid;

## Function to create a Gaussian pyramid

In [6]:
//-------------------------------------------------------
void createGaussianPyramid(const Mat& anOriginalImage,
                           vector<Mat>& aGaussianPyramid,
                           size_t aNumberOfLevels)
//-------------------------------------------------------
{
    // Make sure the pyramid is empty
    aGaussianPyramid.clear();

    // Add the original image
    Mat source = anOriginalImage;
    aGaussianPyramid.push_back(source);

    // Add the other sizes
    for (unsigned int i(0); i < aNumberOfLevels - 1; ++i)
    {
        // Compute the new image
        Mat dst;
        pyrDown(source, dst, Size(source.cols / 2, source.rows / 2));

        // Store the new image in the pyramid
        aGaussianPyramid.push_back(dst);

        // The new image becomes the new source
        source = dst;
    }
}


## Function to create a Laplacian pyramid from Gaussian pyramid

In [7]:
//--------------------------------------------------------------
void createLaplacianPyramid(const vector<Mat>& aGaussianPyramid,
                            vector<Mat>& aLaplacianPyramid)
//--------------------------------------------------------------
{
    // Create an empty pyramid
    aLaplacianPyramid.clear();

    // Last image ID to have been processed
    unsigned int last_id(aGaussianPyramid.size());

    // Add the original image
    Mat source = aGaussianPyramid[--last_id];
    aLaplacianPyramid.push_back(source);

    for (unsigned int i = 0; i < aGaussianPyramid.size() - 1; ++i)
    {
        Mat dst;
        pyrUp(source, dst, Size(source.cols*2, source.rows*2));

        // Get the new source from the Gaussian pyramid
        source = aGaussianPyramid[--last_id];

        // Store the new image in the pyramid
        aLaplacianPyramid.push_back(source - dst);
    }
    }

## Synthesis

In [8]:
Mat reconstruct(const vector<Mat>& aLaplacianPyramid, int aLevel)
{
    Mat reconstruction;

    if (aLaplacianPyramid.size())
    {
        for (int i = 0; i < aLaplacianPyramid.size() - aLevel; ++i)
        {
            if (i == 0)
            {
                reconstruction = aLaplacianPyramid[i];
            }
            else
            {
                Mat dst;
                pyrUp(reconstruction, dst, Size(reconstruction.cols * 2, reconstruction.rows * 2));

                reconstruction = dst + aLaplacianPyramid[i];
            }
        }
    }

    return reconstruction;
}

## A function to visualise pyramids

In [9]:
Mat displayPyramid(const std::vector<Mat>& aPyramid)
{
    cv::Scalar background_colour(51, 34, 184, 255);
    cv::Mat window_data(256, 256, CV_8UC3, background_colour);

    if (aPyramid.size())
    {
        int EDGE = 2;
        bool ascending(true);

        Mat tmp;

        if (aPyramid.front().rows > aPyramid.back().rows)
        {
            tmp = aPyramid.front();
            ascending = true;
        }
        else
        {
            tmp = aPyramid.back();
            ascending = false;
        }

        window_data = Mat(tmp.rows * 1.5 + EDGE * 2.0,
                          tmp.cols  + EDGE * (aPyramid.size() + 1),
                          CV_8UC3, 
                          background_colour);
        
        int x_offset(EDGE);
        int y_offset(EDGE);

        for (unsigned int i = 0; i < aPyramid.size(); ++i)
        {
            if (ascending)
            {
                tmp = aPyramid[i];
            }
            else
            {
                tmp = aPyramid[aPyramid.size() - 1 - i];
            }

            normalize(tmp, tmp, 0, 255, NORM_MINMAX, CV_8UC3);
            Mat targetROI = window_data(Rect(x_offset, y_offset, tmp.cols, tmp.rows));

            tmp.copyTo(targetROI);

            if (i == 0)
            {
                y_offset += tmp.rows + EDGE;
            }
            else
            {
                x_offset += tmp.cols + EDGE;
            }
        }
    }

    return window_data;
}

## Read the images

![Orange](../orange.jpg)
![Apple](../apple.jpg)

In [10]:
/// Test image - Make sure it s divisible by 2^{n}
Mat orange = imread("../orange.jpg", IMREAD_COLOR);
if (!orange.data)
{
    throw " No data! -- Exiting the program ";
}

In [11]:
/// Test image - Make sure it s divisible by 2^{n}
Mat apple = imread("../apple.jpg", IMREAD_COLOR);
if (!apple.data)
{
    throw "No data! -- Exiting the program";
}

Convert the data to floating-point numbers. It's important for the pixel-wise subtraction.

In [12]:
apple.convertTo(apple, CV_32FC3);
orange.convertTo(orange, CV_32FC3);

## Check the image sizes

Same size? The two images MUST have the same size.

In [13]:
if (apple.rows != orange.rows || apple.cols != orange.cols)
{
    throw "The two images don't have the same size -- Exiting the program";
}

Power of two? We create a function that returns 

1. the input if the input is a power of two, or 
2. the next power of two. 

For example:

- `powerOfTwo(0)` returns 1;
- `powerOfTwo(1)` returns 1;
- `powerOfTwo(2)` returns 2;
- `powerOfTwo(3)` returns 4; and
- `powerOfTwo(4)` returns 4.

In [14]:
size_t powerOfTwo(size_t aValue)
{
    size_t i = 1;
    
    while (i < aValue) i *= 2;
    
    return i;
}

Check the new function

In [15]:
for (size_t i = 0; i < 18 ; ++i)
    cout << i << "\t" << powerOfTwo(i) << endl;

0	1
1	1
2	2
3	4
4	4
5	8
6	8
7	8
8	8
9	16
10	16
11	16
12	16
13	16
14	16
15	16
16	16
17	32


It seems to work fine.

Get the image size as a power of two

In [16]:
size_t width = powerOfTwo(apple.cols);
size_t height = powerOfTwo(apple.rows);

The size isn't valid

In [17]:
if (width != apple.cols || height != apple.rows)
{
    throw "The image size must be a power of two. -- Exiting the program";
}

## Create the Gaussian pyramids

In [18]:
size_t levels = 8;

vector<Mat> apple_gaussian_pyramid;
vector<Mat> orange_gaussian_pyramid;

createGaussianPyramid(apple, apple_gaussian_pyramid, levels);
createGaussianPyramid(orange, orange_gaussian_pyramid, levels);

In [19]:
Mat apple_gaussian_pyramid_vis = displayPyramid(apple_gaussian_pyramid);
Mat orange_gaussian_pyramid_vis = displayPyramid(orange_gaussian_pyramid);

imwrite("../apple_gaussian_pyramid.png", apple_gaussian_pyramid_vis);
imwrite("../orange_gaussian_pyramid.png", orange_gaussian_pyramid_vis);

![Gaussian pyramid of the apple with 7 levels](../apple_gaussian_pyramid.png)
![Gaussian pyramid of the orange with 7 levels](../orange_gaussian_pyramid.png)

## Create the Laplacian pyramid

In [20]:
vector<Mat> apple_laplacian_pyramid;
vector<Mat> orange_laplacian_pyramid;

createLaplacianPyramid(apple_gaussian_pyramid, apple_laplacian_pyramid);
createLaplacianPyramid(orange_gaussian_pyramid, orange_laplacian_pyramid);

In [21]:
Mat apple_laplacian_pyramid_vis = displayPyramid(apple_laplacian_pyramid);
Mat orange_laplacian_pyramid_vis = displayPyramid(orange_laplacian_pyramid);

imwrite("../apple_laplacian_pyramid.png", apple_laplacian_pyramid_vis);
imwrite("../orange_laplacian_pyramid.png", orange_laplacian_pyramid_vis);

![Laplacian pyramid of the apple with 7 levels](../apple_laplacian_pyramid.png)
![Laplacian pyramid of the orange with 7 levels](../orange_laplacian_pyramid.png)

## Test the synthesis

In [22]:
for (size_t i = 0; i < levels; ++i)
{
    Mat reconstruction = reconstruct(apple_laplacian_pyramid, i);
    reconstruction.convertTo(reconstruction, CV_8UC3);
    
    stringstream file_name;
    file_name << "../apple-synthesis" << i + 1 << ".png";
    imwrite(file_name.str(), reconstruction);
}

## Swap the two halves

Below is a procedure to swap the left half of two images. The images must have the same size.

In [23]:
void swapHalves(Mat& anImage1, Mat& anImage2)
{
    // Check if the two images have the same size
    if (anImage1.cols != anImage2.cols || anImage1.rows != anImage2.rows)
    {
        throw runtime_error("The two images don't have the same size. Cannot swap halves");
    }
    
    // Get the half width
    int half_width = anImage1.cols / 2;

    // Define the left half as a rectangle
    // Not sure why CLing is not happy
    //Rect rectangle1(0, 0, half_width, aImage1.rows);
    Rect rectangle;
    rectangle.x = 0;
    rectangle.y = 0;
    rectangle.width = half_width;
    rectangle.height = anImage1.rows;

    // Move the left half of aImage1 into aImage2, and
    // the left half of aImage2 into aImage1
    cv::Mat ROI1 = anImage1(rectangle);
    cv::Mat ROI2 = anImage2(rectangle);

    Mat temp = ROI1.clone();

    ROI2.copyTo(ROI1);
    temp.copyTo(ROI2);
}

For each level, swap the two ha;ves.

In [24]:
for (size_t i = 0; i < apple_laplacian_pyramid.size(); ++i)
{
    swapHalves(apple_laplacian_pyramid[i], orange_laplacian_pyramid[i]);
}

In [25]:
Mat apporange_laplacian_pyramid_vis = displayPyramid(apple_laplacian_pyramid);
Mat oranapple_laplacian_pyramid_vis = displayPyramid(orange_laplacian_pyramid);

imwrite("../apporange_laplacian_pyramid.png", apporange_laplacian_pyramid_vis);
imwrite("../oranapple_laplacian_pyramid.png", oranapple_laplacian_pyramid_vis);

![Laplacian pyramid of the apporange with 7 levels](../apporange_laplacian_pyramid.png)
![Gaussian pyramid of the oranapple_ with 7 levels](../oranapple_laplacian_pyramid.png)

## Synthesis

In [26]:
Mat reconstruction1 = reconstruct(apple_laplacian_pyramid, 0);
Mat reconstruction2 = reconstruct(orange_laplacian_pyramid, 0);

reconstruction1.convertTo(reconstruction1, CV_8UC3);
reconstruction2.convertTo(reconstruction2, CV_8UC3);

imwrite("../apporange-synthesis.png", reconstruction1);
imwrite("../oranapple-synthesis.png", reconstruction2);

![](../apporange-synthesis.png)
![](../oranapple-synthesis.png)